# Labels Notebook – Datadog + OpenSSF Only  
## Build `labels_version.csv` and `labels_package.csv`

This notebook creates **two label tables**:

- `data/meta/labels_version.csv` – version-level labels  
- `data/meta/labels_package.csv` – package-level labels  

Data sources:

- **Datadog malicious-software-packages-dataset**  
  `data/external/malicious-software-packages-dataset/`
- **OpenSSF malicious-packages (OSV format)**  
  `data/external/ossf-malicious-packages/`

No registry metadata or other features are used here. The goal is **only** to build the
two CSV label files consumed by later notebooks (`version_diff_live_registries_v3`,
`new_features_v4`).


In [1]:
# Cell 2 – Imports & project paths

from pathlib import Path
import json

import numpy as np
import pandas as pd

# Detect project root robustly:
# - If current dir has 'data', assume that's the project root (e.g. running at Jupyter_v2/)
# - Else, if parent has 'data', assume parent is project root (e.g. running from notebooks/)
NOTEBOOK_DIR = Path.cwd()
if (NOTEBOOK_DIR / "data").is_dir():
    PROJECT_ROOT = NOTEBOOK_DIR
elif (NOTEBOOK_DIR.parent / "data").is_dir():
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    PROJECT_ROOT = NOTEBOOK_DIR  # fallback; adjust manually if needed

DATA_DIR = PROJECT_ROOT / "data"
META_DIR = DATA_DIR / "meta"
EXTERNAL_DIR = DATA_DIR / "external"

# External datasets (cloned repos)
DATADOG_ROOT = EXTERNAL_DIR / "malicious-software-packages-dataset"
OSSF_ROOT = EXTERNAL_DIR / "ossf-malicious-packages"

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DATA_DIR     :", DATA_DIR)
print("META_DIR     :", META_DIR)
print("EXTERNAL_DIR :", EXTERNAL_DIR)
print("DATADOG_ROOT :", DATADOG_ROOT)
print("OSSF_ROOT    :", OSSF_ROOT)


PROJECT_ROOT : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2
DATA_DIR     : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data
META_DIR     : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\meta
EXTERNAL_DIR : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\external
DATADOG_ROOT : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\external\malicious-software-packages-dataset
OSSF_ROOT    : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\external\ossf-malicious-packages


In [3]:
# Cell 3 – Helper to load Datadog manifests into a label table

def load_datadog_manifest_labels(root: Path) -> pd.DataFrame:
    """
    Parse Datadog malicious-software-packages-dataset manifests.

    Source layout (relative to `root`):
      samples/pypi/manifest.json
      samples/npm/manifest.json

    Manifest semantics (per Datadog README):
      - Each manifest is a mapping:
          { "<package_name>": null | ["1.0.0", "1.0.1", ...], ... }
      - If entry is null:
          The package has malicious intent; *all* versions are considered malicious.
          We treat this as a **package-level** label (match_scope = "package").
      - If entry is a list of versions:
          Only those versions are malicious (match_scope = "version").

    Returns
    -------
    DataFrame with columns:
      - ecosystem: 'pypi' or 'npm'
      - package_name
      - version (may be None for package-scope entries)
      - match_scope: 'version' or 'package'
      - label_source: 'datadog-manifest'
      - is_malicious_datadog: bool
    """
    rows = []

    for eco in ("pypi", "npm"):
        manifest_path = root / "samples" / eco / "manifest.json"
        if not manifest_path.exists():
            print(f"[Datadog] No manifest for ecosystem '{eco}' at {manifest_path}")
            continue

        with manifest_path.open("r", encoding="utf-8") as f:
            manifest = json.load(f)

        for pkg_name, entry in manifest.items():
            # Package-intent malicious: all versions considered malicious
            if entry is None:
                rows.append(
                    dict(
                        ecosystem=eco,
                        package_name=pkg_name,
                        version=None,
                        match_scope="package",
                        label_source="datadog-manifest",
                    )
                )
                continue

            # Some variants may store a dict with 'versions', or a raw list
            if isinstance(entry, dict):
                versions = entry.get("versions") or []
            else:
                versions = entry or []

            for ver in versions:
                rows.append(
                    dict(
                        ecosystem=eco,
                        package_name=pkg_name,
                        version=str(ver),
                        match_scope="version",
                        label_source="datadog-manifest",
                    )
                )

    if not rows:
        print("[Datadog] No labels found – returning empty table.")
        return pd.DataFrame(
            columns=[
                "ecosystem",
                "package_name",
                "version",
                "match_scope",
                "label_source",
                "is_malicious_datadog",
            ]
        )

    dd_df = pd.DataFrame(rows)
    dd_df["is_malicious_datadog"] = True

    # Normalise basic types
    dd_df["ecosystem"] = dd_df["ecosystem"].str.lower()
    dd_df["package_name"] = dd_df["package_name"].astype(str)

    print("Datadog labels shape:", dd_df.shape)
    return dd_df


# Actually load Datadog labels (or empty if repo missing)
if DATADOG_ROOT.exists():
    datadog_labels = load_datadog_manifest_labels(DATADOG_ROOT)
else:
    print(f"[Datadog] Root not found, using empty label table: {DATADOG_ROOT}")
    datadog_labels = pd.DataFrame(
        columns=[
            "ecosystem",
            "package_name",
            "version",
            "match_scope",
            "label_source",
            "is_malicious_datadog",
        ]
    )

datadog_labels.head()


Datadog labels shape: (9028, 6)


,ecosystem,package_name,version,match_scope,label_source,is_malicious_datadog
0,pypi,0wneg,None,package,datadog-manifest,True
1,pypi,1337test,None,package,datadog-manifest,True
2,pypi,282828282828282828,None,package,datadog-manifest,True
3,pypi,EZBEAMER,None,package,datadog-manifest,True
4,pypi,EZBEAMSW,None,package,datadog-manifest,True


In [9]:
# Cell 4 – Helper to load OSV malicious packages as labels (recursive search)

def load_osv_malicious_labels(root: Path) -> pd.DataFrame:
    """
    Parse OSV JSON files from the OpenSSF `malicious-packages` repo and
    build version-level labels for npm + PyPI packages.

    Expected layout (standard GitHub clone):

        root/
          osv/
            malicious/
              npm/...
              pypi/...
              <other ecosystems>...

    This function:
      * Finds all *.json files recursively under the `osv` directory.
      * Keeps only affected entries for ecosystems 'pypi' and 'npm'.
      * Uses explicit 'versions' lists; range-only entries are skipped.
    """
    # Prefer the 'osv' subdirectory if it exists; otherwise search from root
    osv_dir = root / "osv"
    if not osv_dir.exists():
        print(f"[OSV] 'osv' directory not found at {osv_dir}, searching from root instead.")
        osv_dir = root

    # RECURSIVE search – this is what fixes the "Found 0 OSV JSON files" issue
    json_paths = sorted(osv_dir.rglob("*.json"))
    print(f"[OSV] Found {len(json_paths)} OSV JSON files under {osv_dir}.")

    if not json_paths:
        print("[OSV] No versioned labels found – returning empty table.")
        return pd.DataFrame(
            columns=[
                "ecosystem",
                "package_name",
                "version",
                "osv_id",
                "osv_published",
                "osv_modified",
                "osv_summary",
                "is_malicious_osv",
            ]
        )

    rows = []
    for path in json_paths:
        try:
            data = json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            print(f"[OSV] Failed to parse {path.relative_to(root)}: {e}")
            continue

        osv_id = data.get("id")
        osv_published = data.get("published")
        osv_modified = data.get("modified")
        osv_summary = data.get("summary")

        affected = data.get("affected") or []
        for aff in affected:
            pkg = aff.get("package") or {}
            eco = (pkg.get("ecosystem") or "").lower()
            name = pkg.get("name") or ""

            # Only care about PyPI + npm here
            if eco not in ("pypi", "npm") or not name:
                continue

            versions = aff.get("versions") or []
            if not versions:
                # Some OSV entries specify only ranges; skip those in this notebook
                continue

            for ver in versions:
                rows.append(
                    dict(
                        ecosystem=eco,
                        package_name=name,
                        version=str(ver),
                        osv_id=osv_id,
                        osv_published=osv_published,
                        osv_modified=osv_modified,
                        osv_summary=osv_summary,
                        is_malicious_osv=True,
                    )
                )

    if not rows:
        print("[OSV] No versioned labels extracted – returning empty table.")
        return pd.DataFrame(
            columns=[
                "ecosystem",
                "package_name",
                "version",
                "osv_id",
                "osv_published",
                "osv_modified",
                "osv_summary",
                "is_malicious_osv",
            ]
        )

    osv_df = pd.DataFrame(rows)

    # Normalise dtypes
    osv_df["ecosystem"] = osv_df["ecosystem"].str.lower()
    osv_df["package_name"] = osv_df["package_name"].astype(str)
    osv_df["version"] = osv_df["version"].astype(str)
    osv_df["is_malicious_osv"] = osv_df["is_malicious_osv"].fillna(True).astype(bool)

    print("OSV labels shape:", osv_df.shape)
    return osv_df


# Actually load OSV labels (or empty if repo missing)
if OSSF_ROOT.exists():
    osv_labels = load_osv_malicious_labels(OSSF_ROOT)
else:
    print(f"[OSV] Root not found, using empty label table: {OSSF_ROOT}")
    osv_labels = pd.DataFrame(
        columns=[
            "ecosystem",
            "package_name",
            "version",
            "osv_id",
            "osv_published",
            "osv_modified",
            "osv_summary",
            "is_malicious_osv",
        ]
    )

osv_labels.head()


[OSV] Found 124 OSV JSON files under c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\external\ossf-malicious-packages\osv.
OSV labels shape: (175, 8)


,ecosystem,package_name,version,osv_id,osv_published,osv_modified,osv_summary,is_malicious_osv
0,pypi,0-8,0.0.0.1,MAL-2025-2925,2025-03-28T13:05:11Z,2025-03-28T13:05:11Z,Malicious code in 0-8 (PyPI),True
1,pypi,0x000testqwe,5.20.4,MAL-2025-6428,2025-07-31T19:13:52Z,2025-07-31T19:13:52Z,Malicious code in 0x000testqwe (PyPI),True
2,pypi,123bla,0.0.1,MAL-2024-10238,2024-10-27T13:55:45Z,2024-10-27T13:55:45Z,Malicious code in 123bla (PyPI),True
3,pypi,1337test,1,MAL-2024-4720,2024-06-25T13:32:02Z,2024-10-24T01:01:57Z,Malicious code in 1337test (PyPI),True
4,pypi,18923aa,4.96,MAL-2024-4721,2024-06-25T13:32:02Z,2024-10-24T01:01:57Z,Malicious code in 18923aa (PyPI),True


In [10]:
# Cell 5 – Build version-level and package-level label tables

# --- 5.1 Split Datadog labels into version vs package scope ---

dd_version_scope = datadog_labels[datadog_labels.get("match_scope") == "version"].copy()
dd_package_scope = datadog_labels[datadog_labels.get("match_scope") == "package"].copy()

if not dd_version_scope.empty:
    dd_version_scope = dd_version_scope[
        ["ecosystem", "package_name", "version", "is_malicious_datadog"]
    ]

if not dd_package_scope.empty:
    dd_package_scope = (
        dd_package_scope[["ecosystem", "package_name"]]
        .drop_duplicates()
        .assign(is_malicious_datadog_package=True)
    )

print("dd_version_scope   :", dd_version_scope.shape)
print("dd_package_scope   :", dd_package_scope.shape)
print("osv_labels         :", osv_labels.shape)

# --- 5.2 Build labels_version (version-level) ---

version_frames = []

if not dd_version_scope.empty:
    version_frames.append(dd_version_scope)

if not osv_labels.empty:
    osv_subset_cols = [
        "ecosystem",
        "package_name",
        "version",
        "osv_id",
        "osv_published",
        "osv_modified",
        "osv_summary",
        "is_malicious_osv",
    ]
    present_cols = [c for c in osv_subset_cols if c in osv_labels.columns]
    version_frames.append(osv_labels[present_cols])

if version_frames:
    labels_version = pd.concat(version_frames, ignore_index=True)
else:
    labels_version = pd.DataFrame(
        columns=[
            "ecosystem",
            "package_name",
            "version",
            "is_malicious_datadog",
            "osv_id",
            "osv_published",
            "osv_modified",
            "osv_summary",
            "is_malicious_osv",
            "is_malicious_any_version",
        ]
    )

if not labels_version.empty:
    # Normalise core columns
    labels_version["ecosystem"] = labels_version["ecosystem"].str.lower()
    labels_version["package_name"] = labels_version["package_name"].astype(str)
    labels_version["version"] = labels_version["version"].astype(str)

    for col in ["is_malicious_datadog", "is_malicious_osv"]:
        if col in labels_version.columns:
            labels_version[col] = labels_version[col].fillna(False).astype(bool)
        else:
            labels_version[col] = False

    labels_version["is_malicious_any_version"] = (
        labels_version[["is_malicious_datadog", "is_malicious_osv"]]
        .fillna(False)
        .any(axis=1)
    )

    # Aggregate duplicates (same ecosystem / package / version from multiple sources)
    agg_dict = {
        "is_malicious_datadog": "max",
        "is_malicious_osv": "max",
        "osv_id": "first",
        "osv_published": "first",
        "osv_modified": "first",
        "osv_summary": "first",
        "is_malicious_any_version": "max",
    }
    agg_columns = {c: agg for c, agg in agg_dict.items() if c in labels_version.columns}

    labels_version = (
        labels_version
        .groupby(["ecosystem", "package_name", "version"], as_index=False)
        .agg(agg_columns)
    )

print("labels_version shape:", labels_version.shape)
labels_version.head()


dd_version_scope   : (481, 4)
dd_package_scope   : (8547, 3)
osv_labels         : (175, 8)
labels_version shape: (656, 10)


,ecosystem,package_name,version,is_malicious_datadog,is_malicious_osv,osv_id,osv_published,osv_modified,osv_summary,is_malicious_any_version
0,npm,@ahmedhfarag/ngx-perfect-scrollbar,20.0.20,True,False,None,None,None,None,True
1,npm,@ahmedhfarag/ngx-virtual-scroller,4.0.4,True,False,None,None,None,None,True
2,npm,@art-ws/common,2.0.28,True,False,None,None,None,None,True
3,npm,@art-ws/config-eslint,2.0.4,True,False,None,None,None,None,True
4,npm,@art-ws/config-ts,2.0.7,True,False,None,None,None,None,True


In [11]:
# Cell 6 – Build package-level label table

pd.set_option("future.no_silent_downcasting", True)

# 1) Derive package-level labels from version-level table
if not labels_version.empty:
    labels_package_base = (
        labels_version
        .groupby(["ecosystem", "package_name"], as_index=False)
        .agg(
            has_malicious_version=("is_malicious_any_version", "max"),
            has_malicious_datadog_version=("is_malicious_datadog", "max"),
            has_malicious_osv_version=("is_malicious_osv", "max"),
        )
    )
else:
    labels_package_base = pd.DataFrame(
        columns=[
            "ecosystem",
            "package_name",
            "has_malicious_version",
            "has_malicious_datadog_version",
            "has_malicious_osv_version",
        ]
    )

# 2) Merge in Datadog "malicious-intent" package-scope labels
if not dd_package_scope.empty:
    labels_package = labels_package_base.merge(
        dd_package_scope,
        on=["ecosystem", "package_name"],
        how="outer",
    )
else:
    labels_package = labels_package_base.copy()

# 3) Normalise boolean columns and final package label
for col in [
    "has_malicious_version",
    "has_malicious_datadog_version",
    "has_malicious_osv_version",
    "is_malicious_datadog_package",
]:
    if col not in labels_package.columns:
        labels_package[col] = False

    s = labels_package[col].fillna(False)
    labels_package[col] = s.astype(bool)

labels_package["label_malicious"] = (
    labels_package["has_malicious_version"]
    | labels_package["is_malicious_datadog_package"]
)

labels_package["label"] = np.where(
    labels_package["label_malicious"],
    "malicious",
    "unlabeled_benign_default",
)

print("labels_version shape :", labels_version.shape)
print("labels_package shape :", labels_package.shape)

print("\nSample package-level rows:")
display_cols_package = [
    col
    for col in [
        "ecosystem",
        "package_name",
        "has_malicious_version",
        "is_malicious_datadog_package",
        "label_malicious",
        "label",
    ]
    if col in labels_package.columns
]
labels_package[display_cols_package].head()


labels_version shape : (656, 10)
labels_package shape : (8855, 8)

Sample package-level rows:


,ecosystem,package_name,has_malicious_version,is_malicious_datadog_package,label_malicious,label
0,npm,000webhost-admin,False,True,True,malicious
1,npm,000webhost-analytics,False,True,True,malicious
2,npm,000webhost-api,False,True,True,malicious
3,npm,000webhost-api-client,False,True,True,malicious
4,npm,000webhost-auth,False,True,True,malicious


In [12]:
# Cell 7 – Save label tables to data/meta

META_DIR.mkdir(parents=True, exist_ok=True)

LABELS_VERSION_PATH = META_DIR / "labels_version_v1.csv"
LABELS_PACKAGE_PATH = META_DIR / "labels_package_v1.csv"

labels_version.to_csv(LABELS_VERSION_PATH, index=False)
labels_package.to_csv(LABELS_PACKAGE_PATH, index=False)

print("Saved version-level labels to :", LABELS_VERSION_PATH)
print("Saved package-level labels to :", LABELS_PACKAGE_PATH)


Saved version-level labels to : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\meta\labels_version_v1.csv
Saved package-level labels to : c:\Users\becai\Desktop\CSI 4900 Jupyter\Jupyter_v2\data\meta\labels_package_v1.csv


In [13]:
# Cell 8 – Quick sanity checks (optional)

print("labels_version head:")
display(labels_version.head())

print("\nlabels_package head:")
display(labels_package.head())

print("\nLabel counts (package-level):")
print(labels_package["label"].value_counts(dropna=False))


labels_version head:


,ecosystem,package_name,version,is_malicious_datadog,is_malicious_osv,osv_id,osv_published,osv_modified,osv_summary,is_malicious_any_version
0,npm,@ahmedhfarag/ngx-perfect-scrollbar,20.0.20,True,False,None,None,None,None,True
1,npm,@ahmedhfarag/ngx-virtual-scroller,4.0.4,True,False,None,None,None,None,True
2,npm,@art-ws/common,2.0.28,True,False,None,None,None,None,True
3,npm,@art-ws/config-eslint,2.0.4,True,False,None,None,None,None,True
4,npm,@art-ws/config-ts,2.0.7,True,False,None,None,None,None,True



labels_package head:


,ecosystem,package_name,has_malicious_version,has_malicious_datadog_version,has_malicious_osv_version,is_malicious_datadog_package,label_malicious,label
0,npm,000webhost-admin,False,False,False,True,True,malicious
1,npm,000webhost-analytics,False,False,False,True,True,malicious
2,npm,000webhost-api,False,False,False,True,True,malicious
3,npm,000webhost-api-client,False,False,False,True,True,malicious
4,npm,000webhost-auth,False,False,False,True,True,malicious



Label counts (package-level):
label
malicious    8855
Name: count, dtype: int64
